Implementation of 5 data QC steps identified as : https://app.clickup.com/9009146577/v/dc/8cftaph-127/8cftaph-287

In [4]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import time
import plotly.express as pe
import plotly.offline as pyo
import plotly.graph_objects as go

from datetime import datetime


os.chdir('/Users/gaurav/UAH/temperature_modelling/')

from lib import dataprocess as dp
from lib import crowdqc_test as qc
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# dp.process_downloaded_data('Madison','wunderground','pws','2021')
df = pd.read_csv('data/processed_data/Madison_2022/master_Madison_pws_.csv')
df = dp.process_wunder(df)


### Step 1 : Clean data with more than 35% missing observations

In [6]:
print(f'Old Stats : Num of stations {len(df.station.unique())}, Total rows {len(df)}')
soc = (df.groupby(['station']).count()/86)['temperature']
valid_stations = soc.iloc[np.where(soc > 35)].index.tolist()
df = df[df['station'].isin(valid_stations)]

print(f'New Stats : Num of stations {len(df.station.unique())}, Total rows {len(df)}')

Old Stats : Num of stations 182, Total rows 1207770
New Stats : Num of stations 147, Total rows 1149262


### Step 2 : Filling null values

Idea is : Fill null values with interpolated values for now and later, if there is some discrepancy in data, qc1,qc2 steps will handle it.

In [7]:
df['temperature'] = df['temperature'].interpolate(method='linear',limit_direction='both')

### Step 3 : QC paper implementations


In [8]:
temp_field = 'temperature'

#Getting indexes with invalid temperature values
index_ = qc.level1_check(df,temp_field)
df = df[~df.index.isin(index_)]

print('Following indexes removed after level 1 check: ',index_)

QC check 1 failed : Temperature exceeds given range
#########################
56.0
-74.0
Following indexes removed after level 1 check:  Int64Index([986116, 986117, 986118, 986119, 986120, 986121, 986122, 986123,
            986124, 986125, 986126, 986161, 986162, 986163, 986164, 986165,
            986166, 986167, 986593, 986594, 986595, 986596, 986597],
           dtype='int64')


In [9]:
time_resolution = qc.level2_check(df,temp_field,'beg_time')


QC Check 2 passed : Spike Test
#########################
